# SMK Dataset: k-Stability Experiments
Experiments to show that there exist features that are k-unstable.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio

from sklearn.linear_model import Lasso
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler

## Load dataset
Load the SMK dataset. There are 187 instances, 19993 features and a binary outcome variable in the set \{1, 2\}. We standardize by subtracting the mean and dividing by the std. deviation.

The SMK data set contains 187 smokers either with or without lung cancer. Each of the 19993 features represents one gene in each smoker, and the outcome variable represents lung cancer.

In [2]:
mat_file = sio.loadmat('./data/SMK_CAN_187.mat')
X = mat_file['X']
y = mat_file['Y']
print(np.shape(X))
print(np.shape(y))
print(X[:5])
print(y[:5])

(187, 19993)
(187, 1)
[[10.697  5.345  7.142 ...  5.209  7.569  6.748]
 [10.424  5.463  7.246 ...  4.215  8.01   6.859]
 [10.374  5.482  7.815 ...  4.249  8.144  6.928]
 [10.5    5.584  7.404 ...  4.582  7.526  6.966]
 [ 9.82   5.612  7.511 ...  4.619  8.983  7.247]]
[[1]
 [1]
 [1]
 [1]
 [1]]


In [3]:
# normalize the dataset 
scaler = StandardScaler()
X = scaler.fit_transform(X)
y_mean = np.mean(y)
y_std = np.std(y)
y = (y - y_mean)/y_std
print(X[:5])
print(y[:5])

[[ 1.22079395 -1.2754427  -1.17996428 ...  1.36877403 -0.850822
  -0.92880521]
 [ 0.57804618 -0.80677576 -0.74875782 ... -1.43159776  0.22351176
  -0.55180162]
 [ 0.46032681 -0.73131244  1.61043906 ... -1.3358104   0.54995331
  -0.31744803]
 [ 0.75697963 -0.32619355 -0.0936557  ... -0.39765768 -0.95557563
  -0.18838374]
 [-0.84400383 -0.21498445  0.34998941 ... -0.29341848  2.59386719
   0.76601274]]
[[-1.03816077]
 [-1.03816077]
 [-1.03816077]
 [-1.03816077]
 [-1.03816077]]


## LASSO Model
We use Lasso Regression to find the k-unstable features. Following optimization problem:

$$\min_{\theta} \frac{1}{n} \cdot \|Y - X\theta|_2^2 + \frac{2\lambda}{n} \cdot \|\theta\|_1$$

Because Lasso induces sparsity on the resulting $\theta$ vector of the model (typically used for feature selection), we find all the features that get set to 0, and call this our "target set." We aim to loop through the features in our "target set" and find the smallest value of $k$ (the number of poison rows) that adds that specific feature to $\theta$. 

sklearn's implementation of LASSO, that we're using here, is the following equivalent optimization problem, with hyperparameter $\alpha$:

$$\min_{\theta} \frac{1}{2n} \|Y - X \theta\|_2^2 + \alpha \|\theta\|_1$$

In [4]:
# TODO: get a more systematic way to find alpha
alpha_val = 0.05
lasso = Lasso(alpha=alpha_val) # default: alpha = 1.0
lasso_coef = lasso.fit(X,y).coef_
print(np.where(lasso_coef > 0)[0])
print("Number of positive nonzero entries at alpha={}: {}".format(alpha_val, len(lasso_coef[lasso_coef > 0])))

[  298  1653  1768  2343  2424  2595  2709  2714  2784  3054  3562  3790
  3918  4155  4735  5496  5701  6577  6757  8627  8679  8889  8905  9032
  9145  9382  9698 10309 10526 10597 10650 10738 11083 11146 11148 11161
 11249 11331 11563 11583 11652 11776 12384 12396 12411 12502 12641 13491
 13575 13664 14301 14773 15368 15700 15705 16260 16397 16542 16785 17071
 17842 19625 19757]
Number of positive nonzero entries at alpha=0.05: 63


In [5]:
# Initialize the support and set of poisoning targets
support = np.where(abs(lasso_coef) > 1.e-6)[0]
target_set = np.where(abs(lasso_coef) <= 1e-6)[0]
print(target_set)
print(len(target_set))

[    0     1     2 ... 19990 19991 19992]
19886


### Because SMK is too big, take 5,000 random features instead

In [6]:
target_set = np.random.choice(target_set, 5000, replace=False)
print(len(target_set))
print(target_set)

5000
[16224 11557 10115 ...  8053  8907 17693]


### Poisoning Attack
We follow the poisoning attack proposed in **Theorem 4.5** and **Construction 4.6**. That is, for some $k$, the number of posion rows, we generate:

$$X_p = \left[ {\begin{array}{cccc}
   0 & \dots & 1 & \dots & 0 \\
   \vdots & \ddots & \vdots & \ddots & \vdots \\
   0 & \dots & 1 & \dots & 0 \\
  \end{array} } 
  \right]
$$

for the features of our attack, and 

$$Y_p = \left[ {\begin{array}{c}
   1 \\
   \vdots \\
   1
  \end{array} } 
  \right]
$$

for the outcome variable of our attack. Then, for $X_0$ and $Y_0$ our original dataset, we append the poison rows $X_p$ and $Y_p$ to get:

$$\left[ {\begin{array}{c|c}
   X_0 & Y_0 \\
   X_p & Y_p \\
  \end{array} } 
  \right]
$$

In [7]:
# takes the number of rows to poison, fits a LASSO on the poisoned dataset
def poisoned_lasso(target, poison_rows, X, y): 
    # Generate poison vectors
    X_poison = np.zeros(shape=(poison_rows, len(X[0])))
    X_poison[:, target] += 1 # switch last column to 1's
    y_poison = np.ones(poison_rows)        

    # Poisoned datasets
    X_poisoned = np.vstack([X, X_poison])
    y_poisoned = np.append(y, y_poison)
    
    poisoned_lasso = lasso.fit(X_poisoned,y_poisoned)
    return poisoned_lasso.coef_[target], poisoned_lasso.coef_

def plot_features_k_full(target_k_dict):
    mean_k = np.asarray(list(target_k_dict.values())).mean()
    plt.figure(figsize=(20, 3))
    
    # Set color of minimum feature
    colors = ['c']*len(list(target_k_dict.values()))
    colors[0] = 'r'
            
    plt.bar(range(len(target_k_dict)), list(target_k_dict.values()), align='center', color=colors)
    x_ticks = []
    #for key in list(target_k_dict.keys()):
    #    x_ticks.append(str(key))
    #plt.xticks(range(len(target_k_dict)), x_ticks, rotation=90)
    plt.xticks([], [])
    plt.axhline(y=mean_k,linewidth=1, color='r')
    plt.xlabel("Features Not in Supp")
    plt.ylabel("Minimum k to add to Supp")
    plt.show()
    
def plot_features_k_random_subset(target_k_dict, size):
    mean_k = np.asarray(list(target_k_dict.values())).mean()
    plt.figure(figsize=(20, 3))
    
    # Set color of minimum feature
    colors = ['c']*len(list(target_k_dict.values()))
    colors[0] = 'r'
    
    min_feature = list(target_k_dict.keys())[0]
    min_value = list(target_k_dict.values())[0]
    
    # Randomly sample 'size' number of features to graph (NOT including the minimum)
    target_keys = list(target_k_dict.keys()).copy()
    target_keys.remove(min_feature)
    subset_features = np.random.choice(target_keys, size, replace=False)
    subset_values = []
    for feature in subset_features:
        subset_values.append(target_k_dict[feature])
        
    # Append min feature and value to start of lists
    subset_features = np.insert(subset_features, 0, min_feature)
    subset_values = np.insert(subset_values, 0, min_value)
    
    plt.bar(range(len(subset_features)), subset_values, align='center', color=colors)
    x_ticks = []
    for key in subset_features:
        x_ticks.append(str(key))
    plt.xticks(range(len(subset_features)), x_ticks, rotation=90)
    plt.axhline(y=mean_k,linewidth=1, color='r')
    plt.title("SMK k-stability")
    plt.xlabel("Features Not in Supp")
    plt.ylabel("Minimum k to add to Supp")
    plt.show()

### Experiment Setup
We aim to find features that exhibit k-unstability, as compared to the other features in our dataset. To do this, we iteratively attempt the poison attack on each feature $i$, varying $k$ (the number of rows in our poison attack) until we find the smallest $k$ that adds $i$ to the sparse LASSO vector ("adding" meaning making its value positive). Do this for each feature. 

Because there are 19,993 features, we do this on random subsets of 50 features from the 19,993. We take 5 of these subsets and check the results.

Conjecture: there exist some features that are much more k-unstable than others, i.e. the $k$ needed to poison that feature is substantially lower than the average value of $k$ across all features.

## Binary Search

In [ ]:
# Binary Search for k-values
# can't really do this because, at some point, they just all go to 0 again. 
# one possible solution if we really need binary search: try over a set of possible K_RANGES
K_RANGE = 1000
k_values = list(range(K_RANGE))
target_k_dict = dict()

for target in target_set:
    first = 0
    last = K_RANGE - 1
    found = False
    while(first <= last and not found):
        mid = (first + last)//2
        coef, _ = poisoned_lasso(target, mid, X, y)
        if(abs(coef) > 1e-6):
            prev_coef, _ = poisoned_lasso(target, mid - 1, X, y)
            if(abs(prev_coef) <= 1e-6):
                found = True
                target_k_dict[target] = mid
            else:
                last = mid - 1
        else:
            first = mid + 1
            
sorted_target_k_dict = {k: v for k, v in sorted(target_k_dict.items(), key=lambda item: item[1])}

In [ ]:
# Plot the results for all the features
plot_features_k_full(sorted_target_k_dict)

In [ ]:
# Plot the results for random subset of size
size = 50
plot_features_k_random_subset(sorted_target_k_dict, size)

In [ ]:
# Just for plotting the final images
import pickle
with open('SMK_dict.pkl', 'wb') as file:
    pickle.dump(sorted_target_k_dict, file)